In [ ]:
%load_ext autoreload

%autoreload 2
%matplotlib inline

In [ ]:
import os
import sys 
import configparser
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import configparser
from os.path import join as oj
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression
sys.path.insert(0, "../src")
import severson_data
import seaborn as sns
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from cycler import cycler

In [ ]:
colours = ['#208F90', '#8F2317','#17608F','#8F5F17','#f2f3f4']
golden_ratio =1.618

sns.set_palette(sns.color_palette(colours))

sns.set(style="ticks", rc={"lines.linewidth": 2.5},font_scale=1.3 )

colors = cycler('color',
                colours)
plt.rc('axes', facecolor='#FFFFFF', edgecolor='#000000',
       axisbelow=True, grid=True, prop_cycle=colors)

plt.rc('font', family='serif', size = 20)
plt.rc('text', usetex=False)
plt.rc('axes', labelsize=20)
plt.rc('axes', titlesize=20)

plt.rc('xtick', labelsize='x-small')
plt.rc('ytick', labelsize='x-small')
plt.rc('xtick', labelsize='x-small')
plt.rc('ytick', labelsize='x-small')

In [ ]:
config = configparser.ConfigParser()
config.read('../config.ini');
result_path =config['PATHS']['result_path'] 

In [ ]:
model_path  = '../models/final_models_nifl'
fig_path =config['PATHS']['figure_path'] 
fnames = sorted([oj(model_path, fname) for fname in os.listdir(model_path) if "pkl" in fname]) 
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 


In [ ]:
results_all = pd.concat(results_list, axis=1).T.infer_objects()

results_all.experiment_name.unique()


In [ ]:
results_all = results_all.rename(columns={'train_percentage': '% Dataset used', 
                                  'rmse_state_test': 'RMSE (Modeled)',  
                                  'rmse_life_test': 'RMSE (Prediction)',
                                         'sequence_length': '# Cycles', 
                                         'use_cycle_counter': 'use_num_cycles'});
results_all.use_num_cycles = results_all.use_num_cycles.map({0: 'No cycle count', 1: 'With cycle count'})

In [ ]:
 if 'data_dict' not in locals(): # just takes a lot of time
    data_path = config['DATASET']['severson_path'] 
    bat_dicts = severson_data.load_data_single(data_path)
    data_dict = {**bat_dicts[0], **bat_dicts[1],}

# Make plot of all

In [ ]:
with open(oj(fig_path, 'uncertainEnsembleLSTM_100.pickle'), "rb") as output_file:
    pkl_dict = pkl.load( output_file)
num_bats = 21


seq_length = 100
x_pred = pkl_dict['fifty_percentile']
all_lower_quantile = pkl_dict['lower_percentile']
all_upper_quantile = pkl_dict['upper_percentile']
max_used_steps = 3500
fig, axes = plt.subplots(ncols=5, nrows =3, figsize = (30, 15),sharex = True,sharey = True)
linewidth = 4

for i, ax in enumerate(fig.get_axes()):
    if i >= num_bats:
        break
    
    ax.plot(x_preprocessed[i], c = colours[1], label = 'Data',linewidth = linewidth)

        
    lower_limit = all_lower_quantile[i]
    upper_limit = all_upper_quantile[ i]
    ax.fill_between(np.arange(max_used_steps), upper_limit, lower_limit , facecolor=colours[0], alpha=0.2)
    ax.plot(x_pred[i,], c= colours[0], label = 'Predicted',linewidth = linewidth)

    
    ax.set_ylim(.8*1.1+.002,1.1)
    ax.set_xticks([0, 1000,2000],['0', '1k','2k'],fontsize=35 )
    ax.set_yticks([.9,1, 1.1],[.9,1, 1.1],fontsize=35 )

    
    ax.set_xlim(0,2300)
    ax.axvline(x = seq_length , linestyle = "--", c = 'k', label = 'Used for\nprediction')

axes[-1,2].set_xlabel("Cycle number",fontsize=50 )
axes[1,0].set_ylabel("Q (Ah)",fontsize=50 )
axes[0,-1].legend(loc =1, prop={'size': 30})
plt.tight_layout()

plt.savefig(oj(fig_path,"uncertainEnsembleLSTM_100.pdf".format(seq_length, )) )

In [ ]:
x_pred.shape

In [ ]:
with open(oj(fig_path, 'agedUncertainEnsembleLSTM_100.pickle'), "rb") as output_file:
    pkl_dict = pkl.load( output_file)
num_bats = x_pred.shape[0]


seq_length = 100
x_pred = pkl_dict['fifty_percentile']
all_lower_quantile = pkl_dict['lower_percentile']
all_upper_quantile = pkl_dict['upper_percentile']
max_used_steps = 3500
fig, axes = plt.subplots(ncols=5, nrows =3, figsize = (30, 15),sharex = True,sharey = True)
linewidth = 4

for i, ax in enumerate(fig.get_axes()):
    if i >= num_bats:
        break
    
    ax.plot(x_preprocessed[i], c = colours[1], label = 'Data',linewidth = linewidth)

        
    lower_limit = all_lower_quantile[i]
    upper_limit = all_upper_quantile[ i]
    ax.fill_between(np.arange(max_used_steps), upper_limit, lower_limit , facecolor=colours[0], alpha=0.2)
    ax.plot(x_pred[i,], c= colours[0], label = 'Predicted',linewidth = linewidth)

    
    ax.set_ylim(.8*1.1+.002,1.1)
    ax.set_xticks([0, 1000,2000],['0', '1k','2k'],fontsize=35 )
    ax.set_yticks([.9,1, 1.1],[.9,1, 1.1],fontsize=35 )

    
    ax.set_xlim(0,2300)
    ax.axvline(x = seq_length , linestyle = "--", c = 'k', label = 'Used for\nprediction')

axes[-1,2].set_xlabel("Cycle number",fontsize=50 )
axes[1,0].set_ylabel("Q (Ah)",fontsize=50 )
axes[0,-1].legend(loc =1, prop={'size': 30})
plt.tight_layout()

# plt.savefig(oj(fig_path,"uncertainEnsembleLSTM_100.pdf".format(seq_length, )) )

# Create figures

In [ ]:
config = configparser.ConfigParser()
config.read('../config.ini');
result_path =config['PATHS']['result_path'] 

In [ ]:
cycle_files = [x for x in os.listdir(result_path) if 'NumCycles' in x]
plot_df = pd.DataFrame(columns = ['# Cycles', 'RMSE', 'RMSE (Aged)'])
for cur_file in cycle_files:
    seq_len = int(''.join([(s) for s in cur_file if s.isdigit()])) #lmao, regex
    cur_result_dict = pkl.load(  open( os.path.join(result_path, cur_file), "rb" ) )
    for i in range(len(cur_result_dict['test'])):
        plot_df = plot_df.append({'# Cycles':seq_len, 'RMSE':cur_result_dict['test'][i], 'RMSE (Aged)':cur_result_dict['aged'][i]}, ignore_index=True)
#         plot_df = plot_df.append({'# Cycles':seq_len, 'RMSE':cur_result_dict['test'][i], 'RMSE (Aged)':cur_result_dict['aged'][i]}, ignore_index=True)
#         pd.concat([plot_df, 
#                    {'# Cycles':seq_len, 'RMSE':cur_result_dict['test'][i], 'RMSE (Aged)':cur_result_dict['aged'][i]}])
        
fig, axes = plt.subplots(ncols=1, figsize=(5,5))

my_plot = sns.lineplot(ax = axes,
    data=plot_df,
    x="# Cycles", y="RMSE"#,  join=False 
)



#     if(i%2 != 0): labels[i] = '' # skip even labels
#     else: labels[i] = str(int(float(l.get_text())))
# axes.set_xticklabels(labels, rotation=30) # set new labels
plt.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "sequence_vs_predicted.pdf"))
fig.savefig(oj(config['PATHS']['figure_path'], "sequence_vs_predicted.png"))

In [ ]:
cycle_files = [x for x in os.listdir(result_path) if 'NumCycles' in x]
plot_df = pd.DataFrame(columns = ['# Cycles', 'RMSE', 'RMSE (Aged)'])
for cur_file in cycle_files:
    seq_len = int(''.join([(s) for s in cur_file if s.isdigit()])) #lmao, regex
    cur_result_dict = pkl.load(  open( os.path.join(result_path, cur_file), "rb" ) )
    for i in range(len(cur_result_dict['test'])):
        plot_df = plot_df.append({'# Cycles':seq_len, 'RMSE':cur_result_dict['test'][i], 'RMSE (Aged)':cur_result_dict['aged'][i]}, ignore_index=True)
fig, axes = plt.subplots(ncols=1, figsize=(5,5))

my_plot = sns.pointplot(ax = axes,
    data=plot_df,
    x="# Cycles", y="RMSE",  join=False 
)

labels = axes.get_xticklabels() # get x labels
for i,l in enumerate(labels):


    if(i%2 != 0): labels[i] = '' # skip even labels
    else: labels[i] = str(int(float(l.get_text())))
axes.set_xticklabels(labels, rotation=30) # set new labels
plt.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "sequence_vs_predicted.pdf"))
fig.savefig(oj(config['PATHS']['figure_path'], "sequence_vs_predicted.png"))

In [ ]:
fig, axes = plt.subplots(ncols=1, figsize=(5,5))

my_plot = sns.pointplot(ax = axes,
    data=plot_df,
    x="# Cycles", y="RMSE (Aged)", join=False
)
labels = axes.get_xticklabels() # get x labels
for i,l in enumerate(labels):
    if(i%2 != 0): labels[i] = '' # skip even labels
    else: labels[i] = str(int(float(l.get_text())))
axes.set_xticklabels(labels, rotation=30) # set new labels
plt.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "sequence_vs_predicted_aged.pdf"))
fig.savefig(oj(config['PATHS']['figure_path'], "sequence_vs_predicted_aged.png"))

In [ ]:
x, y, c, var  = severson_data.get_capacity_input({**bat_dicts[0], **bat_dicts[1],**bat_dicts[2]}, num_offset=0, start_cycle = 10, stop_cycle = 100)

#remember to load
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
ax.scatter(c[:84,1], y[:84], label = 'Not calendar-aged ' )
ax.scatter(c[84:,1], y[84:], label = 'Calendar-aged')
ax.legend()
ax.set_ylabel("Lifetime")
ax.set_xlabel("Average C-Rate")
fig.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "calendar_aging_effect.pdf"))
fig.savefig(oj(config['PATHS']['figure_path'], "calendar_aging_effect.png"))

In [ ]:
np.median(y[:84])

In [ ]:
np.mean(y[:84])

In [ ]:
x, y, c, var  = severson_data.get_capacity_input({**bat_dicts[0], **bat_dicts[1],**bat_dicts[2]}, num_offset=0, start_cycle = 10, stop_cycle = 100)

#remember to load
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
ax.scatter(var[:42,1], np.log(y[:42]), label = 'Not calendar-aged ' )
# ax.legend()
ax.set_ylabel("Log(Lifetime)")
ax.set_xlabel("Delta Coulombic efficiency")
fig.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "coulombic.pdf"))
fig.savefig(oj(config['PATHS']['figure_path'], "coulombic.png"))

In [ ]:
cycle_files = [x for x in os.listdir(result_path) if 'NumCycles' in x]
plot_df = pd.DataFrame(columns = ['# Cycles', 'RMSE', 'RMSE (Aged)'])
for cur_file in cycle_files:
    seq_len = int(''.join([(s) for s in cur_file if s.isdigit()])) #lmao, regex
    cur_result_dict = pkl.load(  open( os.path.join(result_path, cur_file), "rb" ) )
    for i in range(len(cur_result_dict['test'])):
        plot_df = plot_df.append({'# Cycles':seq_len, 'RMSE':cur_result_dict['test'][i], 'RMSE (Aged)':cur_result_dict['aged'][i]}, ignore_index=True)
fig, axes = plt.subplots(ncols=2, figsize=(9,5))
filtered_plotdf_no_temp = plot_df[(plot_df['# Cycles'] < 50) +  (plot_df['# Cycles'] > 70)]
line_width = 3
axes[0].errorbar(filtered_plotdf_no_temp.groupby('# Cycles').mean().index, 
                 filtered_plotdf_no_temp.groupby('# Cycles').mean().RMSE, 
                 yerr=filtered_plotdf_no_temp.groupby('# Cycles').std().RMSE, 
                 lw = line_width,
                 ms  = 3*line_width,
                 c = colours[0], 
                 fmt='o')
filtered_plotdf = plot_df[(plot_df['# Cycles']>= 50) *  (plot_df['# Cycles'] <=70)]

axes[0].errorbar(filtered_plotdf.groupby('# Cycles').mean().index, 
                 filtered_plotdf.groupby('# Cycles').mean().RMSE, 
                 yerr=filtered_plotdf.groupby('# Cycles').std().RMSE,
                 fmt='*',
                 ms  = 3*line_width,
                 lw = line_width,
                 c= colours[1], 
                 label = 'Temperature\naberration')


axes[0].annotate('Temperature\naberration', (60, 360),
            xytext=(0.55, 0.75), textcoords='axes fraction',
            arrowprops=dict( shrink=0.1, facecolor ='white'),
            fontsize=10,
            horizontalalignment='right', verticalalignment='top')


axes[0].set_ylabel("RMSE (# Cycles)")
axes[0].set_xlabel("# Input cycles")



cycle_files = [x for x in os.listdir(result_path) if 'NumBats' in x]
plot_df = pd.DataFrame(columns = ['# Batteries', 'RMSE (# Cycles)', 'RMSE (Aged)'])
for cur_file in cycle_files:
    seq_len = (42*int(''.join([(s) for s in cur_file if s.isdigit()]))/100) #lmao, regex
    cur_result_dict = pkl.load(  open( os.path.join(result_path, cur_file), "rb" ) )
    for i in range(len(cur_result_dict['test'])):
        plot_df = plot_df.append({'# Batteries':seq_len, 'RMSE (# Cycles)':cur_result_dict['test'][i], 'RMSE (Aged)':cur_result_dict['aged'][i]}, ignore_index=True)


my_plot = sns.lineplot(ax = axes[1],
    data=plot_df,
    x="# Batteries", y="RMSE (# Cycles)" #, join = False
)
axes[0].title.set_text("A")
axes[1].title.set_text("B")


plt.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "combined_rmse_vs.pdf"))
fig.savefig(oj(config['PATHS']['figure_path'], "combined_rmse_vs.png"))

In [ ]:
seq_length = 100
result_nets = pkl.load(  open( os.path.join(result_path, "results_dict_{:}.pkl".format(seq_length)), "rb" ) )
train_idxs , val_idxs,test_idxs= severson_data.get_split(len(result_nets['y']), seed =42)


markers = ['o', 'v', '*']
linewidth = 2
markersize = 7
label_size=10
idx_dict = {'train':train_idxs, 'val':val_idxs, 'test':test_idxs}
y_dict = pkl.load(  open( os.path.join(result_path, "results_dict_{:}.pkl".format(seq_length)), "rb" ) )
true_vals = y_dict['y']
y_pred = y_dict['y_pred']
fig, ax = plt.subplots(figsize = (10,5))
ax.plot([0,2500],[0, 2500], c= 'k',linewidth=linewidth, linestyle = ':' )
for i,partition in enumerate(idx_dict.keys()):

    ax.errorbar(true_vals[idx_dict[partition]],#
                y_pred[:,idx_dict[partition]].mean(axis=0), 
                y_pred[:,idx_dict[partition]].std(axis=0), 
                c= colours[i], marker = markers[i], 
                linestyle = 'None', label = partition,markersize = markersize)
    
ax.set_ylabel("Predicted lifetime")
ax.set_xlabel("Observed lifetime")

ax.legend(fontsize = label_size,bbox_to_anchor=(1, 1))
plt.tight_layout()
# plt.savefig(oj(fig_path,"Predictions_vs_true_{0!s}.pdf".format(seq_length)) )
# plt.savefig(oj(fig_path,"Predictions_vs_true_{0!s}.png".format(seq_length)) )

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
ax.hist(y[:84], bins =20, label = "Not calendar-aged", density = True)
ax.hist(y[84:], bins =10, label = "Calendar-aged", density = True, alpha = .7)
ax.legend()
ax.set_ylabel("# Batteries")
ax.set_xlabel("Lifetime")
fig.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "lifetimes.pdf"))
fig.savefig(oj(config['PATHS']['figure_path'], "lifetimes.png"))



In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
ax.hist(y[train_idxs], bins =10, label = "Train", density = True)

ax.hist(y[test_idxs], bins =10, label = "Test", density = True, alpha = .7)
# ax.hist(y[84:], bins =10, label = "Calendar-aged", density = False, alpha = .7)
ax.legend()
ax.set_ylabel("# Batteries (Density)")
ax.set_xlabel("Lifetime")
fig.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "lifetimes_train.pdf"))
fig.savefig(oj(config['PATHS']['figure_path'], "lifetimes_train.png"))


# load figures


In [ ]:
import pickle
seq_length = 100
unc_ensemble_dict = pkl.load(open(os.path.join(fig_path, "uncertainEnsembleLSTM_{0!s}.pickle".format(seq_length, )),"rb") )
aged_ensemble_dict = pkl.load(open(os.path.join(fig_path, "agedUncertainEnsembleLSTM_{0!s}.pickle".format(seq_length, )),"rb") )

In [ ]:
ncols, nrows = 6,3
fig, axes = plt.subplots(ncols=ncols, nrows =nrows, figsize = (20, 10))
linewidth = 4
max_used_steps = (unc_ensemble_dict['fifty_percentile']).shape[1]
all_upper_quantile = unc_ensemble_dict['upper_percentile']
all_lower_quantile = unc_ensemble_dict['lower_percentile']

mean = unc_ensemble_dict['fifty_percentile']
for i, ax in enumerate(fig.get_axes()):


    ax.plot(unc_ensemble_dict['true_sequence '][i], c = colours[1], label = 'Data',linewidth = linewidth)

    ax.fill_between(np.arange(max_used_steps), all_upper_quantile[i], all_lower_quantile[i] , facecolor=colours[0], alpha=0.2)
    ax.plot(mean[i], c= colours[0], label = 'Predicted',linewidth = linewidth)
    ax.set_yticks([.8*1.1,.9, 1.0, 1.1])
    if i%ncols!=0:
        ax.set_yticklabels([],)

    ax.set_ylim(.8*1.1+.002,1.1)


    ax.set_xlim(0,2500)# np.maximum(y[used_idxs[i]], (all_outputs[model_idx, i,:] < 1e-3).argmax())+50)
    ax.axvline(x = seq_length , linestyle = "--", c = 'k', label = 'Used for\nprediction')
axes[1,0].set_ylabel('Capacity', fontsize = 40)
axes[-1,3].set_xlabel('Cycles', fontsize = 40, loc = 'left')
axes[-1,-1].legend(loc =1)
plt.tight_layout()
# plt.savefig(oj(fig_path,"uncertainEnsembleLSTM_{0!s}.png".format(seq_length, )) )

In [ ]:
ncols, nrows = 6,3
fig, axes = plt.subplots(ncols=ncols, nrows =nrows, figsize = (20, 10))
linewidth = 4
max_used_steps = (aged_ensemble_dict['fifty_percentile']).shape[1]
all_upper_quantile = aged_ensemble_dict['upper_percentile']
all_lower_quantile = aged_ensemble_dict['lower_percentile']

mean = aged_ensemble_dict['fifty_percentile']
for i, ax in enumerate(fig.get_axes()):


    ax.plot(aged_ensemble_dict['true_sequence '][i], c = colours[1], label = 'Data',linewidth = linewidth)

    ax.fill_between(np.arange(max_used_steps), all_upper_quantile[i], all_lower_quantile[i] , facecolor=colours[0], alpha=0.2)
    ax.plot(mean[i], c= colours[0], label = 'Predicted',linewidth = linewidth)
    ax.set_yticks([.8*1.1,.9, 1.0, 1.1])
    if i%ncols!=0:
        ax.set_yticklabels([],)

    ax.set_ylim(.8*1.1+.002,1.1)


    ax.set_xlim(0,2500)# np.maximum(y[used_idxs[i]], (all_outputs[model_idx, i,:] < 1e-3).argmax())+50)
    ax.axvline(x = seq_length , linestyle = "--", c = 'k', label = 'Used for\nprediction')
axes[1,0].set_ylabel('Capacity', fontsize = 40)
axes[-1,3].set_xlabel('Cycles', fontsize = 40, loc = 'left')
axes[-1,-1].legend(loc =1)
plt.tight_layout()
plt.savefig(oj(fig_path,"agedUncertainEnsembleLSTM_{0!s}.png".format(seq_length, )) )

# Letter to the editor

In [ ]:
seq_length = 40
unc_ensemble_dict = pkl.load(open(os.path.join(fig_path, "uncertainEnsembleLSTM_{0!s}.pickle".format(seq_length, )),"rb") )
aged_ensemble_dict = pkl.load(open(os.path.join(fig_path, "agedUncertainEnsembleLSTM_{0!s}.pickle".format(seq_length, )),"rb") )
ncols, nrows = 6,3
fig, axes = plt.subplots(ncols=3, nrows =1, figsize = (20, 6))
linewidth = 4
max_used_steps = (unc_ensemble_dict['fifty_percentile']).shape[1]
all_upper_quantile = unc_ensemble_dict['upper_percentile']
all_lower_quantile = unc_ensemble_dict['lower_percentile']

mean = unc_ensemble_dict['fifty_percentile']
my_idxes = 3,4, 9
for i, ax in enumerate(fig.get_axes()):


    ax.plot(unc_ensemble_dict['true_sequence '][my_idxes[i]], c = colours[1], label = 'Data',linewidth = linewidth)

    ax.fill_between(np.arange(max_used_steps), all_upper_quantile[my_idxes[i]], all_lower_quantile[my_idxes[i]] , facecolor=colours[0], alpha=0.2)
    ax.plot(mean[my_idxes[i]], c= colours[0], label = 'Predicted',linewidth = linewidth)
    ax.set_yticks([.8*1.1,.9, 1.0, 1.1])
    if i%ncols!=0:
        ax.set_yticklabels([],)

    ax.set_ylim(.8*1.1+.002,1.1)


    ax.set_xlim(0,2400)# np.maximum(y[used_idxs[i]], (all_outputs[model_idx, i,:] < 1e-3).argmax())+50)
    ax.axvline(x = seq_length , linestyle = "--", c = 'k', label = 'Used for\nprediction')
axes[0].set_ylabel('Capacity', fontsize = 40)
axes[-2].set_xlabel('Cycles', fontsize = 40, loc = 'center')
axes[-1].legend(loc =1)
plt.tight_layout()
plt.savefig(oj(fig_path,"editor_letter{0!s}.png".format(seq_length, )) )

In [ ]:
cycle_files = [x for x in os.listdir(result_path) if 'NumCycles' in x]
plot_df = pd.DataFrame(columns = ['# Cycles', 'RMSE', 'RMSE (Aged)'])
for cur_file in cycle_files:
    seq_len = int(''.join([(s) for s in cur_file if s.isdigit()])) #lmao, regex
    cur_result_dict = pkl.load(  open( os.path.join(result_path, cur_file), "rb" ) )
    for i in range(len(cur_result_dict['test'])):
        plot_df = plot_df.append({'# Cycles':seq_len, 'RMSE':cur_result_dict['test'][i], 'RMSE (Aged)':cur_result_dict['aged'][i]}, ignore_index=True)
fig, axes = plt.subplots(ncols=1, figsize=(5,5))
filtered_plotdf_no_temp = plot_df[(plot_df['# Cycles'] < 50) +  (plot_df['# Cycles'] > 70)]
line_width = 3



cycle_files = [x for x in os.listdir(result_path) if 'NumBats' in x]
plot_df = pd.DataFrame(columns = ['# Batteries', 'RMSE (# Cycles)', 'RMSE (Aged)'])
for cur_file in cycle_files:
    seq_len = (42*int(''.join([(s) for s in cur_file if s.isdigit()]))/100) #lmao, regex
    cur_result_dict = pkl.load(  open( os.path.join(result_path, cur_file), "rb" ) )
    for i in range(len(cur_result_dict['test'])):
        plot_df = plot_df.append({'# Batteries':seq_len, 'RMSE (# Cycles)':cur_result_dict['test'][i], 'RMSE (Aged)':cur_result_dict['aged'][i]}, ignore_index=True)


my_plot = sns.lineplot(ax = axes,
    data=plot_df,
    x="# Batteries", y="RMSE (# Cycles)" #, join = False
)



plt.tight_layout()
fig.savefig(oj(config['PATHS']['figure_path'], "rmse_LE.pdf"), dpi =200)
fig.savefig(oj(config['PATHS']['figure_path'], "rmse_LE.png"),dpi =200)

In [ ]:
fig, ax = plt.subplots()
ax = sns.violinplot(x=42 * ['Training'] + 21 * ["Test"], y=np.concatenate((y[train_idxs],y[test_idxs],)) )
ax.set_ylabel("Lifetime")

In [ ]:
fig, ax = plt.subplots()
ax = sns.violinplot(x=84 * ['Not Calendar aged'] + 40 * ["Calendar-aged"], y=y, )
ax.set_ylabel("Lifetime")

# Gradients

In [ ]:
import pickle
gradient_dict = pkl.load(open(os.path.join(result_path, "gradients.pickle"),"rb") )
aged_gradient = pkl.load(open(os.path.join(result_path, "aged_gradients.pickle"),"rb") )

In [ ]:
aged_gradient['y_test']

In [ ]:
aged_grads_dying = aged_gradient['aged_grads_dying']
all_grads_dying = aged_gradient['all_grads_dying']

In [ ]:
all_grads_dying.mean()

In [ ]:
filter_battery_eol = 700
interval = 95
upper_limit = 100 - (100 - interval) / 2
lower_limit = 0 + (100 - interval) / 2
linewidth = 4.5
fontsize = 20

In [ ]:
show_idxs = np.where(aged_gradient['y_test'] > filter_battery_eol)[0]
mean_plot = (
    all_grads_dying[show_idxs, 4:-1].sum(axis=1)
    / all_grads_dying[show_idxs].sum(axis=1)
).mean(axis=0)
std_plot = (
    all_grads_dying[show_idxs, 4:-1].sum(axis=1)
    / all_grads_dying[show_idxs].sum(axis=1)
).std(axis=0)



plt.fill_between(
    np.arange(400),
    mean_plot + std_plot,
    mean_plot - std_plot,
    facecolor=colours[0],
    alpha=0.2,
)

plt.plot(
    (
        all_grads_dying[show_idxs, 4:-1].sum(axis=1)
        / all_grads_dying[show_idxs].sum(axis=1)
    ).mean(axis=0),
    c=colours[0],
    linewidth=linewidth,
    label="Non aged",
)


mean_plot = (
    aged_grads_dying[:, 4:-1].sum(axis=1) / aged_grads_dying[:].sum(axis=1)
).mean(axis=0)
std_plot = (
    aged_grads_dying[:, 4:-1].sum(axis=1) / aged_grads_dying[:].sum(axis=1)
).std(axis=0)


plt.plot(
    (aged_grads_dying[:, 4:-1].sum(axis=1) / aged_grads_dying[:].sum(axis=1)).mean(
        axis=0
    ),
    c=colours[1],
    linewidth=linewidth,
    label="Aged",
)

plt.fill_between(
    np.arange(400),
    mean_plot + std_plot,
    mean_plot - std_plot,
    facecolor=colours[1],
    alpha=0.2,
)

plt.xticks([50 * x for x in range(9)], [-400 + 50 * x for x in range(9)])

plt.legend()
# plt.title("Relative importance of in-cycle  info \n in the last 300 cycles before death for slow and fast dying")

plt.ylabel("Importance  \n of in-cycle info", fontsize=fontsize)
plt.xlabel("Cycles (from 400 before EOL)", fontsize=fontsize)
plt.ylim(0, 0.3)
plt.tight_layout()
plt.savefig(oj(fig_path, "Sensitivity_analysis_aged_vs_nonaged_eol.pdf"))
plt.savefig(oj(fig_path, "Sensitivity_analysis_aged_vs_nonaged_eol.png"))

In [ ]:
import matplotlib.gridspec as gridspec
fig = plt.figure(tight_layout=True, figsize = (10,10))
gs = gridspec.GridSpec(3, 3)
ax = fig.add_subplot(gs[:, 1])

# ax.fill_between(
#     np.arange(400),
#     mean_plot + std_plot,
#     mean_plot - std_plot,
#     facecolor=colours[0],
#     alpha=0.2,
# )


ax.plot(
    (
        all_grads_dying[show_idxs, 4:-1].sum(axis=1)
        / all_grads_dying[show_idxs].sum(axis=1)
    ).mean(axis=0),
    c=colours[0],
    linewidth=linewidth,
    label="Non aged",
)


ax.plot(
    (aged_grads_dying[:, 4:-1].sum(axis=1) / aged_grads_dying[:].sum(axis=1)).mean(
        axis=0
    ),
    c=colours[1],
    linewidth=linewidth,
    label="Aged",
)

ax.fill_between(
    np.arange(400),
    mean_plot + std_plot,
    mean_plot - std_plot,
    facecolor=colours[1],
    alpha=0.2,
)

# ax.set_xticks([50 * x for x in range(9)], [-400 + 50 * x for x in range(9)])

ax.legend()
# plt.title("Relative importance of in-cycle  info \n in the last 300 cycles before death for slow and fast dying")

ax.set_ylabel("Importance  \n of in-cycle info", fontsize=fontsize)
ax.set_xlabel("Cycles (from 400 before EOL)", fontsize=fontsize)
ax.set_ylim(0, 0.3)

# gradient plot

In [ ]:
gradient_dict.keys()

In [ ]:
num_cycles_before_eol = 300
num_ticks = int(num_cycles_before_eol / 50) + 1

In [ ]:
predicted_y = gradient_dict['predicted_y']
arr_of_curves = gradient_dict['arr_of_curves']
all_grads_dying = gradient_dict['all_grads_dying']

linewidth = 3
fontsize = 15

show_idxs = np.where(predicted_y < 600)[0]
# show_idxs = np.where(y[used_idxs] >600)[0]
fig, axes = plt.subplots(
    nrows=4,
    figsize=(8, 5),
    gridspec_kw={"height_ratios": [1, 1.0, 1.0, 1]},
    sharex=True,
)
axes[0].plot(arr_of_curves[0], c=".75", label="Capacity curves")
for i in range(len(predicted_y)):

    axes[0].plot(arr_of_curves[i], c=str(0.75))


# for i in show_idxs:
# axes[1].plot((all_grads_dying[i,0] / all_grads_dying[i].sum(axis=0)), linewidth = linewidth, label = 'Previous capacities')

mean_plot = (
    all_grads_dying[show_idxs, 0] / all_grads_dying[show_idxs].sum(axis=1)
).mean(axis=0)
std_plot = (all_grads_dying[show_idxs, 0] / all_grads_dying[show_idxs].sum(axis=1)).std(
    axis=0
)

# stuff to make plot broken up
for i in range(1, 3):
    axes[i].plot(
        (
            all_grads_dying[show_idxs, 4:-1].sum(axis=1)
            / all_grads_dying[show_idxs].sum(axis=1)
        ).mean(axis=0),
        linewidth=linewidth,
        label="In-cycle info",
    )
    axes[i].plot(
        (all_grads_dying[show_idxs, 0] / all_grads_dying[show_idxs].sum(axis=1)).mean(
            axis=0
        ),
        linewidth=linewidth,
        label="Previous capacities",
    )
    axes[i].plot(
        (
            all_grads_dying[show_idxs, 1:4].sum(axis=1)
            / all_grads_dying[show_idxs].sum(axis=1)
        ).mean(axis=0),
        linewidth=linewidth,
        label="C-Rate",
    )

    axes[i].plot(
        (all_grads_dying[show_idxs, -1] / all_grads_dying[show_idxs].sum(axis=1)).mean(
            axis=0
        ),
        linewidth=linewidth,
        label="Cycle counter",
        linestyle="dashed",
    )


axes[1].set_ylim(0.65, 0.9)
axes[2].set_ylim(0.0, 0.2)

axes[1].spines["bottom"].set_visible(False)
axes[2].spines["top"].set_visible(False)
axes[1].xaxis.tick_top()
axes[1].tick_params(labeltop=False)  # don't put tick labels at the top
axes[2].xaxis.tick_bottom()

d = 0.015  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them
kwargs = dict(transform=axes[1].transAxes, color="k", clip_on=False)
axes[1].plot((-d, +d), (-d, +d), **kwargs)  # top-left diagonal
axes[1].plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

kwargs.update(transform=axes[2].transAxes)  # switch to the bottom axes
axes[2].plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
axes[2].plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal


axes[1].yaxis.set_label_coords(-0.1, 0.3)

# end stuff


axes[1].legend(bbox_to_anchor=(1.1, 1), loc="upper left")
axes[0].legend(bbox_to_anchor=(1.0, 1.03), loc="upper left")
axes[0].set_ylabel("Capacity\n", fontsize=fontsize)
axes[2].set_ylabel("Feature importance\n", fontsize=fontsize)
# axes[3].set_ylabel("Feature\n importance",fontsize = fontsize)
axes[3].set_xlabel("Cycles to EOL", fontsize=fontsize)
axes[1].set_xticks([50 * x for x in range(num_ticks)])
axes[0].set_xticks([50 * x for x in range(num_ticks)])
axes[0].set_xticklabels([], fontsize=fontsize * 0.8)
axes[1].set_xticklabels([], fontsize=fontsize * 0.8)
axes[1].set_xlim(0, num_cycles_before_eol)
axes[0].set_xlim(0, num_cycles_before_eol)
axes[3].set_xlim(0, num_cycles_before_eol)
axes[3].set_ylim(0, 0.14)
axes[3].set_xticklabels(
    [-num_cycles_before_eol + 50 * x for x in range(num_ticks)], fontsize=fontsize * 0.8
)
show_idxs = np.where(predicted_y < 600)[0]
axes[3].plot(
    (all_grads_dying[show_idxs, 5] / all_grads_dying[show_idxs].sum(axis=1)).mean(
        axis=0
    ),
    linewidth=0.7 * linewidth,
    label="Coloumbic Efficiency <600  EOL",
)
# show_idxs = np.where(y[used_idxs] > 600)[0]
axes[3].plot(
    (all_grads_dying[show_idxs, 5] / all_grads_dying[show_idxs].sum(axis=1)).mean(
        axis=0
    ),
    linewidth=0.7 * linewidth,
    label="Coloumbic Efficiency > 600 EOL",
)
show_idxs = np.where(predicted_y < 6000)[0]
# axes[3].plot((all_grads_dying[show_idxs, 6] / all_grads_dying[show_idxs].sum(axis=1)).mean(axis=0),linewidth = .7*linewidth, label = 'Overpotential',)
# axes[3].plot((all_grads_dying[show_idxs, 4] / all_grads_dying[show_idxs].sum(axis=1)).mean(axis=0),linewidth = .7*linewidth, label = 'Difference between charge\nand discharge')

axes[3].legend(bbox_to_anchor=(1.0, 1.03), loc="upper left")

plt.tight_layout()